In [26]:
import os
import requests
import base64
import sys
from dotenv import load_dotenv # type: ignore
import pandas as pd
import numpy as np
from scipy.stats import zscore
import json

load_dotenv()
token = os.getenv("AIPROXY_TOKEN")

In [2]:
def base_request(json):
    url = "https://aiproxy.sanand.workers.dev/openai/v1/chat/completions"
    
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json",
    }
    

    # Make the request
    req = requests.post(url, headers=headers, json=json)
    # Response body
    res = req.json()

    return res

In [3]:
def  request_to_llm_for_metadata(token, system_prompt: str, sample_data: str):
    '''
    Make a request only for analysis prompt
    '''

    url = "https://aiproxy.sanand.workers.dev/openai/v1/chat/completions"
    
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json",
    }
    # Response schema of data
    schema = {
        "type": "object",
        "properties": {
            "columns": {
                "type": "array",
                "description": "List of column names and their respective types",
                "items": {
                    "type": "object",
                    "properties": {
                        "column_name": {"type": "string", "description": "Column name"},
                        "column_type": {"type": "string", "description": "Column type"},
                        
                    },
                    "required": ["column_name", "column_type"],
                },
                "minItems": 1,
            }
        },
        "required": ["columns"],
    }

    functions = [
        {
            "name": "data_schema",
            "description": "Extract column name and the type from a CSV file to process later in python",
            "parameters": schema,
        }
    ]

    # JSON payload
    json = {
        "model": "gpt-4o-mini",
        "functions": functions,
        "function_call": {"name": "data_schema"},
        "messages": [
            {
                "role": "system",
                "content": system_prompt,
            },
            {
                "role": "user",
                "content": sample_data,
            },
        ],
    }

    # Make the request
    req = requests.post(url, headers=headers, json=json)
    # Response body
    res = req.json()

    return res


In [10]:
prompt = (
        "You are an expert at data analysis."
        "You are going to study the sample data provided, return the column names and their respective types in json format;"
        "use function data_schema.The first row will be names and rest data."
        "Be robust, figure out the type by majority vote, cross-check by name of column, ignore empty cells."
        "The data may or may not be clean. Possible types are: 'integer', 'float', 'object', 'boolean', 'date' and 'url'."
    )
    
with open("happiness/happiness.csv", "r") as f:
    sample_data = ''.join([f.readline() for i in range(5)])
    
# Sending request to LLM
res = request_to_llm_for_metadata(token = token, system_prompt = prompt, sample_data = sample_data)
# Response from LLM received

In [11]:
res

{'id': 'chatcmpl-Aez8Lwef19KNbv0T3XPHxUKSQUXUL',
 'object': 'chat.completion',
 'created': 1734331481,
 'model': 'gpt-4o-mini-2024-07-18',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': None,
    'function_call': {'name': 'data_schema',
     'arguments': '{"columns":[{"column_name":"Country name","column_type":"object"},{"column_name":"year","column_type":"integer"},{"column_name":"Life Ladder","column_type":"float"},{"column_name":"Log GDP per capita","column_type":"float"},{"column_name":"Social support","column_type":"float"},{"column_name":"Healthy life expectancy at birth","column_type":"float"},{"column_name":"Freedom to make life choices","column_type":"float"},{"column_name":"Generosity","column_type":"float"},{"column_name":"Perceptions of corruption","column_type":"float"},{"column_name":"Positive affect","column_type":"float"},{"column_name":"Negative affect","column_type":"float"}]}'},
    'refusal': None},
   'logprobs': None,
   'finish_reaso

In [12]:
metadata = json.loads(res['choices'][0]['message']['function_call']['arguments'])['columns']

In [13]:
metadata

[{'column_name': 'Country name', 'column_type': 'object'},
 {'column_name': 'year', 'column_type': 'integer'},
 {'column_name': 'Life Ladder', 'column_type': 'float'},
 {'column_name': 'Log GDP per capita', 'column_type': 'float'},
 {'column_name': 'Social support', 'column_type': 'float'},
 {'column_name': 'Healthy life expectancy at birth', 'column_type': 'float'},
 {'column_name': 'Freedom to make life choices', 'column_type': 'float'},
 {'column_name': 'Generosity', 'column_type': 'float'},
 {'column_name': 'Perceptions of corruption', 'column_type': 'float'},
 {'column_name': 'Positive affect', 'column_type': 'float'},
 {'column_name': 'Negative affect', 'column_type': 'float'}]

In [14]:
def LLM_analysis(prompt, meta):
    json = {
        "model": "gpt-4o-mini",
        "messages": [
            {
                "role": "system",
                "content": prompt,
            },
            {
                "role": "user",
                "content": meta,
            },
        ],
    }
    

    res= base_request(json)
    return res

In [17]:
pr = ("You are given a metadata containing column names and their types. You are expert data analyst."
      "Study the columns and types, perform various complex analysis carefully (for example: correlation matrix or heatmap is error-prone for non-numeric columns)"
      "Dont write code for reading data. I will pass data in variable 'df'."
      "The code should be robust for any type of data sent."
      "Don't print to stdout and charts should be generated in png format instead of printing."
      "Code should not be a function."
      "Return only python code for the analysis no string quoted or backticks, pure python. The code should generate some charts (1 or 3 max)."
      )

response = LLM_analysis(pr, json.dumps(metadata))

In [18]:
print(response['choices'][0]['message']['content'])

import seaborn as sns
import matplotlib.pyplot as plt

# Generate a heatmap for the correlation matrix
correlation_matrix = df.corr()
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap='coolwarm', square=True)
plt.title('Correlation Matrix Heatmap')
plt.savefig('correlation_matrix_heatmap.png')

# Boxplot to visualize the distribution of 'Life Ladder' across different countries
plt.figure(figsize=(15, 8))
sns.boxplot(x='Country name', y='Life Ladder', data=df)
plt.xticks(rotation=90)
plt.title('Boxplot of Life Ladder by Country')
plt.savefig('boxplot_life_ladder_by_country.png')

# Line plot for Life Ladder over the years
plt.figure(figsize=(12, 6))
mean_life_ladder = df.groupby('year')['Life Ladder'].mean().reset_index()
sns.lineplot(x='year', y='Life Ladder', data=mean_life_ladder, marker='o')
plt.title('Average Life Ladder Over Years')
plt.savefig('average_life_ladder_over_years.png')


In [19]:
import chardet

with open('happiness/happiness.csv', 'rb') as f:
    result = chardet.detect(f.read())
    encoding = result['encoding']

results = []
    
df = pd.read_csv('happiness/happiness.csv', encoding=encoding)

In [20]:
exec(response['choices'][0]['message']['content'])

ValueError: could not convert string to float: 'Afghanistan'

In [ ]:
response

In [22]:
url = "https://aiproxy.sanand.workers.dev/openai/v1/chat/completions"
    
headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json",
    }
    

def correct_error_LLM_request(original_prompt, error_in_response_to_correct):
    
    json = {
        "model": "gpt-4o-mini",
        "messages": [
            {
                "role": "system",
                "content": original_prompt,
            },
            {
                "role": "user",
                "content": error_in_response_to_correct,
            },
            {
                "role": "user",
                "content": "Some error occurred. Correct only the erroneous part."
            }
        ]
    }
    res = requests.post(url=url, headers=headers, json=json)
    return res.json()



In [ ]:
try:
    exec(response['choices'][0]['message']['content'])
    flag=1
except:
    corrected_response = correct_error_LLM_request(pr, response['choices'][0]['message']['content'])
    flag=0
    
if flag:
    print("Original Ran...")
else:
    print("Corrected response..")
    print(corrected_response)

In [70]:
print(corrected_response['choices'][0]['message']['content'])

import seaborn as sns
import matplotlib.pyplot as plt

# Generate a heatmap for the correlation matrix, only for numeric columns
correlation_matrix = df.select_dtypes(include=['float64', 'int64']).corr()
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap='coolwarm', square=True)
plt.title('Correlation Matrix Heatmap')
plt.savefig('correlation_matrix_heatmap.png')

# Boxplot to visualize the distribution of 'Life Ladder' across different countries
plt.figure(figsize=(15, 8))
sns.boxplot(x='Country name', y='Life Ladder', data=df)
plt.xticks(rotation=90)
plt.title('Boxplot of Life Ladder by Country')
plt.savefig('boxplot_life_ladder_by_country.png')

# Line plot for Life Ladder over the years
plt.figure(figsize=(12, 6))
mean_life_ladder = df.groupby('year')['Life Ladder'].mean().reset_index()
sns.lineplot(x='year', y='Life Ladder', data=mean_life_ladder, marker='o')
plt.title('Average Life Ladder Over Years')
plt.savefig('average_life_ladder_over_yea

In [ ]:
exec(corrected_response['choices'][0]['message']['content'])

In [39]:

def generate_README(system_prompt, metadata, base64_images_list):
    # Create the API request payload
    json = {
        "model": "gpt-4o-mini",
        "messages": [
            {
                "role" : "system",
                "content": system_prompt,
            },
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": metadata,
                    }
                ],
            }
        ],
    }
    # Adding all images provided to json payload
    for image in base64_images_list:
        image_object = {
            "type": "image_url",
            "image_url": {
                "detail": "low",
                "url": f'data:image/jpeg;base64,{image}'
            }
        }

        json['messages'][1]["content"].append(image_object)


    # Send the request
    response = requests.post(url, headers=headers, json=json)
    return response.json()

In [67]:
# Specify the directory path
directory_path = "/workspaces/tds_project_2/"

# Get a list of all files in the directory
files = os.listdir(directory_path)

# Filter the list to only include PNG files
png_files = [file for file in files if file.endswith(".png")]
final_images = []
# Read each PNG file
for file in png_files:
    # Read the image file as binary data
    with open(file, 'rb') as image_file:
        image_data = image_file.read()

    # Encode the image data to base64
    base64_image = base64.b64encode(image_data).decode('utf-8')
    final_images.append(base64_image)

pr = ("You are an expert data analyst. You are a given the columns and the column types of a csv dataset."
      "You are also given some images of analysis already done by me."
      "Describe in detail: "
      "- The data you received, briefly"
      "- The analysis you carried out"
      "- The insights you discovered"
      "- The implications of your findings (i.e. what to do with the insights)"
      "You need to generate a professional README file ONLY."
)

README_response = generate_README(pr, json.dumps(metadata), final_images)
print(README_response)

{'id': 'chatcmpl-Af0GaiymNF7d7llpavfNNwZB08gC1', 'object': 'chat.completion', 'created': 1734335836, 'model': 'gpt-4o-mini-2024-07-18', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': "# README: Analysis of Country Well-being Indicators \n\n## Overview\nThis README summarizes the analysis of a dataset containing well-being indicators across various countries. The dataset includes multiple variables, such as Life Ladder scores, GDP per capita, social support, and health metrics, recorded over several years. The primary objective of the analysis was to uncover trends and relationships between these indicators and their implications for policy-making.\n\n## Dataset Description\nThe dataset consists of the following columns:\n\n- **Country name**: Identifies each country (object type).\n- **Year**: The year of observation (integer type).\n- **Life Ladder**: A subjective measure of well-being or happiness (float type).\n- **Log GDP per capita**: Natural logarithm of GDP

In [69]:
generated_file = README_response['choices'][0]['message']['content']
# Save the generated image to a file
with open("README.md", "w") as f:
    f.write(generated_file)
    print("Generated README saved...")


Generated README saved...


In [ ]:

#     "base64",

#     "shutil",

In [28]:
# manual analysis
import chardet

def analyze_data(file_path, metadata):
    # Detect encoding
    with open('happiness/happiness.csv', 'rb') as f:
        result = chardet.detect(f.read())
        encoding = result['encoding']

    results = []
    
    df = pd.read_csv(file_path, encoding=encoding)
    category_cols = [x['column_name'] for x in metadata if x['column_type'] == 'object']
    numeric_cols = [x['column_name'] for x in metadata if x['column_type'] in ['integer', 'float']]
    # Find missing values
    missing_values = df.isnull().sum()

    missing = ("Missing Values per Column", missing_values[missing_values > 0])
    results.append(missing)


    # Detect outliers using z-score (for numerical columns only)
    numeric_df = df[numeric_cols]
    z_scores = numeric_df.apply(zscore).abs()
    outliers = (z_scores > 3).sum()

    outlier = ("Outliers per Numeric Column", outliers[outliers > 0])
    results.append(outlier)
    # Summary statistics
    summary = ("Summary Statistics", df.describe())
    results.append(summary)

    return results
